In [1]:
import torch
import torchvision
import torch.nn as nn

from pytorch_revgrad import RevGrad

import copy
import torch.optim as optim
import numpy as np
from tqdm import tqdm
from torchvision.datasets import MNIST
import torch.nn as nn
import torch.nn.functional as F
from torch import einsum

import cv2
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import Compose, ToTensor

from pathlib import Path
from PIL import Image

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

/home/ys5hd/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Params

In [2]:
# Specifiy MNIST and BSDS500 Data Path

DATA_PATH = '/project/GutIntelligenceLab/ys5hd/MNIST/'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Data

In [3]:
class GrayscaleToRgb:
    """Convert a grayscale image to rgb"""
    def __call__(self, image):
        image = np.array(image)
        image = np.dstack([image, image, image])
        return Image.fromarray(image)


class BSDS500(Dataset):
    def __init__(self):
        image_folder = Path(DATA_PATH+'BSDS500/BSR/BSDS500/data/images')
        self.image_files = list(map(str, image_folder.glob('*/*.jpg')))

    def __getitem__(self, i):
        image = cv2.imread(self.image_files[i], cv2.IMREAD_COLOR)
        tensor = torch.from_numpy(image.transpose(2, 0, 1))
        return tensor

    def __len__(self):
        return len(self.image_files)


class MNISTM(Dataset):

    def __init__(self, train=True):
        super(MNISTM, self).__init__()
        self.mnist = datasets.MNIST(DATA_PATH, train=train,
                                    download=False)
        self.bsds = BSDS500()
        # Fix RNG so the same images are used for blending
        self.rng = np.random.RandomState(42)

    def __getitem__(self, i):
        digit, label = self.mnist[i]
        digit = transforms.ToTensor()(digit)
        bsds_image = self._random_bsds_image()
        patch = self._random_patch(bsds_image)
        patch = patch.float() / 255
        blend = torch.abs(patch - digit)
        return blend, label

    def _random_patch(self, image, size=(28, 28)):
        _, im_height, im_width = image.shape
        x = self.rng.randint(0, im_width-size[1])
        y = self.rng.randint(0, im_height-size[0])
        return image[:, y:y+size[0], x:x+size[1]]

    def _random_bsds_image(self):
        i = self.rng.choice(len(self.bsds))
        return self.bsds[i]

    def __len__(self):
        return len(self.mnist)

### Model

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # CNN Feature Extractor
        self.fc_extractor = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=5),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            nn.Dropout(0.25)
        )
        
        
        # Classifier
        self.classifier = nn.Sequential(
             nn.Linear(6400, 32),
             nn.ReLU(),
             nn.Dropout(0.5),
             nn.Linear(32, 10),
        )
        
        # Domain Classifier
        self.domain_classifier = nn.Sequential(
             RevGrad(),
             nn.Linear(6400, 32),
             nn.ReLU(),
             nn.Linear(32, 1),
        )
        
        # Projection Layer
        self.domain_projection = nn.Sequential(
            nn.Linear(6400, 64),
            nn.ReLU(),
        )


    def forward(self, x):
        features = self.fc_extractor(x)
        features = features.view(x.shape[0], -1)
        domain_projection = self.domain_projection(features)
        prediction_class = self.classifier(features)
        prediction_domain = self.domain_classifier(features)
        return prediction_class, prediction_domain, domain_projection

### Data Loader

In [5]:
BATCH_SIZE = 64

In [6]:
source_dataset = MNIST(DATA_PATH, train=True, download=False,
                      transform=Compose([GrayscaleToRgb(), ToTensor()]))
source_loader = DataLoader(source_dataset, batch_size=BATCH_SIZE,
                           shuffle=True, drop_last=True)

source_dataset_test = MNIST(DATA_PATH, train=False, download=False,
                      transform=Compose([GrayscaleToRgb(), ToTensor()]))
source_loader_test = DataLoader(source_dataset_test, batch_size=BATCH_SIZE,
                           shuffle=False)

target_dataset = MNISTM(train=True)
target_loader = DataLoader(target_dataset, batch_size=BATCH_SIZE,
                           shuffle=True, drop_last=True)

target_dataset_test = MNISTM(train=False)
target_loader_test = DataLoader(target_dataset_test, batch_size=BATCH_SIZE,
                           shuffle=False)

### Base Model Training

In [7]:
model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [8]:
epochs = 10
for epoch in range(epochs):
    running_loss = 0

    model.train()
    for images, labels in tqdm(source_loader):
        
        # Cuda
        images = images.to(device)
        labels = labels.to(device)
        
        # Training pass
        optimizer.zero_grad()
        
        output, _, _ = model(images)
        loss = criterion(output.squeeze(), labels)
        
        #This is where the model learns by backpropagating
        loss.backward()
        
        #And optimizes its weights here
        optimizer.step()
        
        running_loss += loss.item()

    print("Epoch {} - Training loss: {}".format(epoch, running_loss/len(source_dataset)))
    
    model.eval()
    correct_prediction = 0
    for images, labels in tqdm(source_loader_test):

        # Cuda
        images = images.to(device)
        labels = labels.to(device)
        
        with torch.no_grad():
            prediction, _, _ = model(images)
            correct_prediction += sum(torch.max(prediction.squeeze(), 1)[1]==labels).item()
    print("Epoch {} - Validation Accuracy: {}".format(epoch, correct_prediction/len(source_dataset_test)))

100%|██████████| 937/937 [00:08<00:00, 108.21it/s]


Epoch 0 - Training loss: 0.008400399645666282


100%|██████████| 157/157 [00:01<00:00, 130.27it/s]


Epoch 0 - Validation Accuracy: 0.9807


100%|██████████| 937/937 [00:08<00:00, 109.63it/s]


...

Epoch 8 - Validation Accuracy: 0.992


100%|██████████| 937/937 [00:08<00:00, 116.43it/s]


Epoch 9 - Training loss: 0.002626434665483733


100%|██████████| 157/157 [00:01<00:00, 132.31it/s]

Epoch 9 - Validation Accuracy: 0.9913


In [9]:
PATH = 'DANN.pt'
torch.save(model.state_dict(), PATH)

#### MNIST-M Test Accuracy

In [10]:
model.eval()
correct_prediction = 0
for images, labels in tqdm(target_loader_test):
    
    # Cuda
    images = images.to(device)
    labels = labels.to(device)
        
    with torch.no_grad():
        prediction, _, _ = model(images)
        correct_prediction += sum(torch.max(prediction.squeeze(), 1)[1]==labels).item()
        
print('Test Accuracy: {}'.format(correct_prediction/len(target_dataset_test)))

100%|██████████| 157/157 [00:31<00:00,  4.95it/s]

Test Accuracy: 0.53


### DANN Model Training

In [17]:
model = Net().to(device)

criterion = nn.CrossEntropyLoss()
criterion_bce = nn.BCEWithLogitsLoss()

optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [18]:
# PATH = 'DANN.pt'
# model.load_state_dict(torch.load(PATH))

In [19]:
domain_iter = iter(target_loader)

epochs = 40
for epoch in range(epochs):
    running_loss_class = 0
    running_loss_domain = 0    

    model.train()
    for images, labels in tqdm(source_loader):
        
        # Cuda
        images = images.to(device)
        labels = labels.to(device)
        
        try:
            images_domain, _ = next(domain_iter)
        except StopIteration:
            domain_iter = iter(target_loader)
            images_domain, _ = next(domain_iter)
        
        images_domain = images_domain.to(device)
        
        # Combine Images
        images = torch.cat((images, images_domain))
        
        # Domain Target
        target_domain = torch.cat((torch.ones(BATCH_SIZE, 1), torch.zeros(BATCH_SIZE, 1))).to(device)        
        
        # Training pass
        optimizer.zero_grad()
                
        output, output_domain, _ = model(images)

        loss_class = criterion(output[:BATCH_SIZE].squeeze(), labels)
        loss_bce = criterion_bce(output_domain, target_domain)
                     
        loss = loss_class + loss_bce
        
        #This is where the model learns by backpropagating
        loss.backward()
        
        #And optimizes its weights here
        optimizer.step()
        
        running_loss_class += loss_class.item()
        running_loss_domain += loss_bce.item()

    print("Epoch {} - Training loss Classifier: {}, Training loss Domain: {}"\
          .format(epoch, running_loss_class/len(source_dataset), running_loss_domain/(2*len(source_dataset))))
    
    model.eval()
    correct_prediction = 0
    for images, labels in tqdm(source_loader_test):

        # Cuda
        images = images.to(device)
        labels = labels.to(device)
        
        with torch.no_grad():
            prediction, _, _ = model(images)
            correct_prediction += sum(torch.max(prediction.squeeze(), 1)[1]==labels).item()
    print("Epoch {} - Validation Accuracy: {}".format(epoch, correct_prediction/len(source_dataset_test)))
    
    model.eval()
    correct_prediction = 0
    for images, labels in tqdm(target_loader_test):

        # Cuda
        images = images.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            prediction, _, _ = model(images)
            correct_prediction += sum(torch.max(prediction.squeeze(), 1)[1]==labels).item()

    print('Test Accuracy: {}'.format(correct_prediction/len(target_dataset_test)))    

100%|██████████| 937/937 [03:24<00:00,  4.59it/s]


Epoch 0 - Training loss Classifier: 0.017157927534977593, Training loss Domain: 0.004095627647141616


100%|██████████| 157/157 [00:01<00:00, 131.65it/s]


Epoch 0 - Validation Accuracy: 0.9302


100%|██████████| 157/157 [00:32<00:00,  4.89it/s]


...

Epoch 39 - Training loss Classifier: 0.005008983059848348, Training loss Domain: 0.0037227531768381597


100%|██████████| 157/157 [00:01<00:00, 132.70it/s]


Epoch 39 - Validation Accuracy: 0.9823


100%|██████████| 157/157 [00:31<00:00,  5.02it/s]

Test Accuracy: 0.7178


#### MNIST-M Test Accuracy

In [20]:
model.eval()
correct_prediction = 0
for images, labels in tqdm(target_loader_test):
    
    # Cuda
    images = images.to(device)
    labels = labels.to(device)
        
    with torch.no_grad():
        prediction, _, _ = model(images)
        correct_prediction += sum(torch.max(prediction.squeeze(), 1)[1]==labels).item()
        
print('Test Accuracy: {}'.format(correct_prediction/len(target_dataset_test)))

100%|██████████| 157/157 [00:31<00:00,  5.01it/s]

Test Accuracy: 0.7112


In [21]:
PATH = 'DANN_adv.pt'
torch.save(model.state_dict(), PATH)

### DANN+JSD Model Training
- We finetune the model pre-trained in previous part.

##### Loss Function

In [7]:
# Infomax Loss =========================================
class GlobalDiscriminator(nn.Module):
    def __init__(self, sz):
        super(GlobalDiscriminator, self).__init__()
        self.l0 = nn.Linear(sz, 512)
        self.l1 = nn.Linear(512, 512)
        self.l2 = nn.Linear(512, 1)

    def forward(self, x1, x2):
        x = torch.cat((x1, x2), dim=1)
        h = F.relu(self.l0(x))
        h = F.relu(self.l1(h))
        return self.l2(h)
    
class DeepInfoMaxLoss(nn.Module):
    def __init__(self, sz=64, type="concat"):
        super().__init__()
        if type=="concat":
            self.global_d = GlobalDiscriminator(sz=2*sz)
        elif type=="dot":
            self.global_d = GlobalDiscriminatorDot(sz=sz)
        else:
            self.global_d = GlobalDiscriminatorConv(sz=2*sz)

    def forward(self, proto_label_pos, proto_label_neg, proto_unlabel_pos):
        Ej = -F.softplus(-self.global_d(proto_unlabel_pos, proto_label_pos)).mean()
        Em = F.softplus(self.global_d(proto_unlabel_pos, proto_label_neg)).mean()
        LOSS = (Em - Ej)
        
        return LOSS

In [8]:
# Load Model
PATH = 'DANN_adv.pt'

model = Net().to(device)
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [9]:
# Initialize
loss_fn = DeepInfoMaxLoss(sz=64, type="concat").to(device)

criterion = nn.CrossEntropyLoss()
criterion_bce = nn.BCEWithLogitsLoss()

optimizer_fn = optim.Adam(loss_fn.parameters(), lr=1e-4)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [10]:
def get_pseudo_prediction(model, images):
    model.eval()
    with torch.no_grad():
        output_prob, _, _ = model(images)
    model.train()
    output_prob = torch.softmax(output_prob, 1)
    return output_prob

### EMA ###
class EMA(object):
    def __init__(self, model, alpha):
        self.step = 0
        self.model = copy.deepcopy(model)
        self.alpha = alpha

    def update(self, model):
        decay = min(1 - 1 / (self.step + 1), self.alpha)
        for ema_param, param in zip(self.model.parameters(), model.parameters()):
            ema_param.data = decay * ema_param.data + (1 - decay) * param.data
        self.step += 1

#### Training Loop

In [11]:
domain_iter = iter(target_loader)
ema_model = EMA(model, 0.99)

epochs = 40
for epoch in range(epochs):
    running_loss_class = 0
    running_loss_mi = 0    
    running_loss_adv = 0
    
    model.train()
    for images, labels in tqdm(source_loader):
        
        # Cuda
        images = images.to(device)
        labels = labels.to(device)
        
        try:
            images_domain, _ = next(domain_iter)
        except StopIteration:
            domain_iter = iter(target_loader)
            images_domain, _ = next(domain_iter)
        
        images_domain = images_domain.to(device)
        
        # Training pass
        optimizer.zero_grad()
        optimizer_fn.zero_grad()
        
        pseudo_output = get_pseudo_prediction(ema_model.model, images_domain)        
                
        output, source_adversarial, source_projection = model(images)
        _, target_adversarial, target_projection = model(images_domain)
        
        source_pos_projection = torch.tensor(()).to(device)
        source_neg_projection = torch.tensor(()).to(device)
        target_pos_projection = torch.tensor(()).to(device)
        target_neg_projection = torch.tensor(()).to(device)                

        pseudo_output = torch.max(pseudo_output, 1)
        filter_index = pseudo_output[0]>0.9

        for cls in pseudo_output[1][filter_index].unique():
            num_max = min(sum(labels==cls), sum(pseudo_output[1][filter_index]==cls), \
                         sum(pseudo_output[1][filter_index]!=cls))

            target_pos_projection = torch.cat((target_pos_projection, \
                                               target_projection[(filter_index) & (pseudo_output[1]==cls)][:num_max]))
            target_neg_projection = torch.cat((target_neg_projection, \
                                               target_projection[(filter_index) & (pseudo_output[1]!=cls)][:num_max]))                        
            source_pos_projection = torch.cat((source_pos_projection, \
                                               source_projection[labels==cls][:num_max]))
            source_neg_projection = torch.cat((source_neg_projection, \
                                               source_projection[labels!=cls][:num_max]))                        
                
        loss_class = criterion(output.squeeze(), labels)
        loss_mi = loss_fn(source_pos_projection, source_neg_projection, target_pos_projection) +\
        loss_fn(target_pos_projection, target_neg_projection, source_pos_projection)

        adversarial_pred = torch.cat((source_adversarial, target_adversarial))
        label_domain = torch.cat((torch.ones(BATCH_SIZE, 1), torch.zeros(BATCH_SIZE, 1))).to(device)        
        loss_domain = criterion_bce(adversarial_pred, label_domain)
                     
        loss = loss_class + loss_mi + loss_domain
        
        #This is where the model learns by backpropagating
        loss.backward()
        
        #And optimizes its weights here
        optimizer.step()
        optimizer_fn.step()
        
        ema_model.update(model)
        
        running_loss_class += loss_class.item()
        running_loss_adv += loss_domain.item()
        running_loss_mi += loss_mi.item()

    print("Epoch {} - Training loss Classifier: {}, Training loss MI: {}, Training Loss Adv: {}"\
          .format(epoch, running_loss_class/len(source_dataset), running_loss_mi, running_loss_adv))
    
    model.eval()
    correct_prediction = 0
    for images, labels in tqdm(source_loader_test):

        # Cuda
        images = images.to(device)
        labels = labels.to(device)
        
        with torch.no_grad():
            prediction, _, _ = model(images)
            correct_prediction += sum(torch.max(prediction.squeeze(), 1)[1]==labels).item()
    print("Epoch {} - Validation Accuracy: {}".format(epoch, correct_prediction/len(source_dataset_test)))

100%|██████████| 937/937 [03:39<00:00,  4.28it/s]


Epoch 0 - Training loss Classifier: 0.004703488431870937, Training loss MI: 1277.1779759526253, Training Loss Adv: 390.79958939552307


100%|██████████| 157/157 [00:01<00:00, 132.03it/s]


Epoch 0 - Validation Accuracy: 0.9834


100%|██████████| 157/157 [00:31<00:00,  4.95it/s]


...

Epoch 39 - Training loss Classifier: 0.002064281786015878, Training loss MI: 50.07603239585296, Training Loss Adv: 473.45588034391403


100%|██████████| 157/157 [00:01<00:00, 133.54it/s]


Epoch 39 - Validation Accuracy: 0.9876


100%|██████████| 157/157 [00:31<00:00,  4.93it/s]

Test Accuracy: 0.8724


In [12]:
model.eval()
correct_prediction = 0
for images, labels in tqdm(target_loader_test):
    
    # Cuda
    images = images.to(device)
    labels = labels.to(device)
        
    with torch.no_grad():
        prediction, _, _ = model(images)
        correct_prediction += sum(torch.max(prediction.squeeze(), 1)[1]==labels).item()

print('Test Accuracy: {}'.format(correct_prediction/len(target_dataset_test)))

100%|██████████| 157/157 [00:30<00:00,  5.08it/s]

Test Accuracy: 0.8762


In [13]:
PATH = 'DANN_adv_mi.pt'
torch.save(model.state_dict(), PATH)